In [ ]:
# coding: utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap

# Input experimental data

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Thermocouple', header = 1)
df2 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Measurements', header = 0)

In [ ]:
df2

In [ ]:
sigma_p_sys = 1 / 10 / 2
sigma_u_sys = 1.0

In [ ]:
def convert_voltage_to_celsius(row):
    return row['dU, мкВ'] / float(df1.iloc[(df1['T, C']-row['T, C']).abs().argsort()[:1]]['мкВ / C'])

In [ ]:
def calculate_temperature_sys_error(row):
    return row['dT, C'] * sigma_u_sys / row['dU, мкВ']

In [ ]:
zero_kelvin = 273
bar = 10e5

In [ ]:
df2['dT, C'] = df2.apply(convert_voltage_to_celsius, axis = 1)
df2['dT_sigma, C'] = df2.apply(calculate_temperature_sys_error, axis = 1)
df2['dP_sigma, бар'] = sigma_p_sys
df2['T, K'] = df2['T, C'] + zero_kelvin
df2['1/T, 1/K'] = 1 / df2['T, K']
df2['dP, Па'] = df2['dP, бар'] * bar

In [ ]:
grouped = df2.groupby(['T, C'])
groups = [group for _, group in grouped]

# Least Squares

In [ ]:
def lstsqb(x, y):
    return (x.multiply(y).mean() - x.mean() * y.mean()) / (x.multiply(x).mean() - x.mean() ** 2)

In [ ]:
def lstsqa(x, y):
    return y.mean() - lstsqb(x, y) * x.mean()

In [ ]:
def lstsqbsem(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt((y.multiply(y).mean() - y.mean() ** 2) / (x.multiply(x).mean() - x.mean() ** 2) - lstsqb(x, y) ** 2)

# Part 1

## General Data

In [ ]:
lines = list()
for i in groups:
    p, v = np.polyfit(i['dP, бар'], i['dT, C'], deg = 1, cov = True)
    lines.append(list([p, np.array([np.sqrt(v[0][0]), np.sqrt(v[1][1])])]))

In [ ]:
lines

# Tables

# Plots

In [ ]:
def plot_group(a, i):
    a.xaxis.set_major_locator(plt.MultipleLocator(0.5))
    a.yaxis.set_major_locator(plt.MultipleLocator(0.5))
    a.xaxis.set_minor_locator(plt.MultipleLocator(0.1))
    a.yaxis.set_minor_locator(plt.MultipleLocator(0.1))
    a.set_ylim(bottom = y_min, top = y_max)
    a.set_xlim(left = x_min, right = x_max)
    a.grid(visible = True, linestyle = '--')
    a.grid(visible = True, linestyle = '--', which = 'minor')
    a.errorbar(groups[i]['dP, бар'], groups[i]['dT, C'], color = colors[i], yerr = groups[i]['dT_sigma, C'], xerr = groups[i]['dP_sigma, бар'], fmt = '.')
    # ax.errorbar(df1.loc[1:, 'dl, см'], df1.loc[1:, 'P, Н'], xerr = df1.loc[1:, 'sigma_l, см'], fmt = '.')
    a.plot(x, lines[i][0][1] + lines[i][0][0] * x, label = 'Линейная модель: $f(\Delta{P}) = a + \Delta{P} \cdot b$', color = colors[i])
    a.plot([], [], ' ', label = '$a = {a:.3f} \pm {da:.3f} \degree C, b = {b:.3f} \pm {db:.3f}$ $\degree C / бар$'.format(a = lines[i][0][1], b = lines[i][0][0], da = lines[i][1][1], db = lines[i][1][0]))
    a.legend()
    a.set_xlabel('$\Delta{P}, бар$')
    a.set_ylabel('$\Delta{T}, C$')

In [ ]:
y_max = 9 / 8 * max(df2.loc[:, 'dT, C'])
x_max = 9 / 8 * max(df2.loc[:, 'dP, бар'])
y_min = 3 / 4 * min(df2.loc[:, 'dT, C'])
x_min = 3 / 4 * min(df2.loc[:, 'dP, бар'])

In [ ]:
x = np.linspace(0, x_max, 1000)

In [ ]:
name = "Set1"
cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
colors = cmap.colors  # type: list

In [ ]:
for i in range(len(groups)):
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
    plot_group(ax, i)
    fig.suptitle('Рис. {}. Зависимость $\Delta{T}(\Delta{P})$ при $T = {t} \degree C$'.format(i + 1, T = '{T}', P = '{P}', t = groups[i]['T, C'].iloc[0]))
    fig.savefig('plots/plot{}.pdf'.format(i))

In [ ]:
array_error = np.array([lines[x][1][0] / bar for x in range(len(lines))])

In [ ]:
array = [[groups[x]['1/T, 1/K'].iloc[0], lines[x][0][0]] for x in range(len(lines))]
p, v = np.polyfit([groups[x]['1/T, 1/K'].iloc[0] for x in range(len(lines))], np.array([lines[x][0][0] / bar for x in range(len(lines))]), deg = 1, cov = True, w = 1 / np.array(array_error))

In [ ]:
v

In [ ]:
x_max = 9 / 8 * max([groups[x]['1/T, 1/K'].iloc[0] for x in range(len(lines))])
y_max = 9 / 8 * max([lines[x][0][0] / bar for x in range(len(lines))])
x_min = 7 / 8 * min([groups[x]['1/T, 1/K'].iloc[0] for x in range(len(lines))])
y_min = 7 / 8 * min([lines[x][0][0] / bar for x in range(len(lines))])

In [ ]:
def plot_final(a):
    a.xaxis.set_major_locator(plt.MultipleLocator(1e-4))
    a.yaxis.set_major_locator(plt.MultipleLocator(0.5e-7))
    a.xaxis.set_minor_locator(plt.MultipleLocator(0.5e-4))
    a.yaxis.set_minor_locator(plt.MultipleLocator(0.1e-7))
    plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    a.set_ylim(bottom = y_min, top = y_max)
    a.set_xlim(left = x_min, right = x_max)
    a.grid(visible = True, linestyle = '--')
    a.grid(visible = True, linestyle = '--', which = 'minor')
    x = np.linspace(x_min, x_max, 1000)
    a.errorbar([groups[x]['1/T, 1/K'].iloc[0] for x in range(len(lines))], np.array([lines[x][0][0] for x in range(len(lines))]) / bar, fmt = '.', color = colors[0], yerr = array_error)
    a.plot(x, p[0] * x + p[1], color = colors[0], label = 'Линейная модель: $\mu_{д-т} = c + d / T$')
    a.plot([], [], ' ', label = '$c = {a:.2e} \pm {da:.2e} \degree K / Па$, $d = {b:.2e} \pm {db:.2e} \degree K^2 / Па$'.format(a = p[1], b = p[0], da = np.sqrt(v[1][1]), db = np.sqrt(v[0][0])))
    a.legend()
    a.set_xlabel('$1 / T, 1 / \degree K$')
    a.set_ylabel('$\mu_{д-т}, \degree K / Па$')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
plot_final(ax)
fig.suptitle('Рис. 6. Зависимость $\mu_{д-т}(1 / T)$')
fig.savefig('plots/plot5.pdf')

# Results

In [ ]:
R = 8.31
c = p[1]
d = p[0]
e = np.sqrt(np.diag(v))
sigma_c = e[1]
sigma_d = e[0]
cp = 7 / 2 * R

In [ ]:
b = - cp * c
sigma_b = b * sigma_c / c

In [ ]:
a = d * R * cp / 2
sigma_a = sigma_d / d * a

In [ ]:
a

In [ ]:
sigma_a

In [ ]:
b

In [ ]:
sigma_b  * 10**6

In [ ]:
T_i = 2 * a / (R * b)
sigma_T_i = T_i * np.sqrt((sigma_a / a) ** 2 + (sigma_b / b) ** 2)

In [ ]:
sigma_T_i

In [ ]:
df2.to_excel('output.xlsx')